In [1]:
import numpy as np
import pandas as pd
import nltk
import math
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import seaborn as sb
sb.set()
import matplotlib.pyplot as plt

%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_hub as hub

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


TensorFlow 2.x selected.


In [8]:
print(tf.__version__)

2.1.0


In [4]:
!pip3 install ktrain

     |████████████████████████████████| 102kB 6.1MB/s 
     |████████████████████████████████| 6.7MB 14.1MB/s 
     |████████████████████████████████| 10.1MB 57.0MB/s 
     |████████████████████████████████| 1.0MB 58.5MB/s 
     |████████████████████████████████| 245kB 64.2MB/s 
     |████████████████████████████████| 1.8MB 51.6MB/s 
     |████████████████████████████████| 491kB 49.8MB/s 
     |████████████████████████████████| 1.0MB 53.6MB/s 
     |████████████████████████████████| 3.8MB 53.1MB/s 
     |████████████████████████████████| 870kB 46.4MB/s 
  Created wheel for ktrain: filename=ktrain-0.9.4-cp36-none-any.whl size=124242 sha256=a91ac4019f8d88b4d108a5f2d0532c30d88083a72a8d804d1397f76f0fe2b627
  Stored in directory: /root/.cache/pip/wheels/72/ff/17/089be78c75eb77a2d6b1905fd537293b7cba9edc16b0e0c880
  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=e90398ffe004ee7505be93ced3f83f3e0e08d2b3896964158f3d08f13cdb63c1
  Stored in directory

In [2]:
import ktrain
from ktrain import text

using Keras version: 2.2.4-tf


In [0]:
#Import data

trainset=pd.read_csv('https://raw.githubusercontent.com/mohitprashant/DisasterTweetClassification/master/train.csv')
testset=pd.read_csv('https://raw.githubusercontent.com/mohitprashant/DisasterTweetClassification/master/test.csv')

In [0]:
#Split trainset

trainset=trainset.sample(frac=1.0)

x_train=pd.DataFrame(trainset[:7000])['text'].tolist()
y_train=pd.DataFrame(trainset[:7000])['target'].tolist()

x_test=pd.DataFrame(trainset[7000:])['text'].tolist()
y_test=pd.DataFrame(trainset[7000:])['target'].tolist()

predictid=pd.DataFrame(testset)['id'].tolist()
predictset=pd.DataFrame(testset)['text'].tolist()

In [5]:
#Process data

(x_train, y_train),(x_test, y_test),preproc=text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       class_names=['1','0'],
                                                                       preprocess_mode='bert',
                                                                       maxlen=400, 
                                                                       max_features=30000)


task: text classification
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


preprocessing test...
language: en


In [7]:
#Create model

mod=text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner=ktrain.get_learner(mod, train_data=(x_train, y_train), batch_size=10)

Is Multi-Label? False
maxlen is 400
done.


In [8]:
#Train for 4 epochs with a learning rate of 3e-5

learner.fit_onecycle(3e-5, 4)



begin training using onecycle policy with max lr of 3e-05...
Train on 7000 samples
Epoch 1/4
7000/7000 [==============================] - 848s 121ms/sample - loss: 0.4668 - accuracy: 0.7873
Epoch 2/4
7000/7000 [==============================] - 839s 120ms/sample - loss: 0.3531 - accuracy: 0.8606
Epoch 3/4
7000/7000 [==============================] - 842s 120ms/sample - loss: 0.2311 - accuracy: 0.9096
Epoch 4/4
7000/7000 [==============================] - 843s 120ms/sample - loss: 0.0970 - accuracy: 0.9640


In [9]:
#Validate

learner.validate(val_data=(x_test,y_test), class_names=['1','0'])

              precision    recall  f1-score   support

           1       0.85      0.86      0.86       349
           0       0.81      0.80      0.81       264

    accuracy                           0.84       613
   macro avg       0.83      0.83      0.83       613
weighted avg       0.83      0.84      0.84       613



array([[301,  48],
       [ 53, 211]])

In [0]:
#Predict

predictor=ktrain.get_predictor(learner.model, preproc)

predictions=[]

In [23]:
predictions=predictor.predict(predictset)

In [0]:
out=pd.DataFrame(list(zip(predictid,predictions)),columns=['id','target'])

In [0]:
#Flip values because they were saved wrongly

for i in range(len(out)):
  if(out.at[i,'target']=='1'):
    out.at[i,'target']='0'
  else:
    out.at[i,'target']='1'

#Save solution

out.to_csv('solution2.csv',index=False)

In [0]:
from google.colab import files
files.download("solution2.csv")